In [2]:
import datetime
import pytz
import zipfile
import os
import pandas as pd

In [4]:
city_name = input("Enter the city name: ")
city_name = city_name.capitalize()

# Specify the path to the zip file you want to unzip
zip_file_path = f'/home/elshan/Projects/GTFS/gtfs_zips/{city_name}_gtfs.zip'

# Specify the directory where you want to extract the contents
extract_path = f'/home/elshan/Projects/GTFS/gtfs_unziped_data/{city_name}'

# Open the zip file for reading
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents to the specified directory
    zip_ref.extractall(extract_path)

print(f'Unzipped {zip_file_path} to {extract_path}')


Unzipped /home/elshan/Projects/GTFS/gtfs_zips/Paris_gtfs.zip to /home/elshan/Projects/GTFS/gtfs_unziped_data/Paris


In [6]:
city_name = input("Enter the city name: ")
city_name = city_name.capitalize()

# Define the folder containing the txt files
folder_path = f'/home/elshan/Projects/GTFS/gtfs_unziped_data/{city_name}'

# Define a mapping from file names to DataFrame names
file_mapping = {
    "stop_times.txt": "stop_times_df",
    "stops.txt": "stops_df",
    "trips.txt": "trips_df",
    "routes.txt": "routes_df"
}

# Initialize a dictionary to store the DataFrames
dataframes = {}

# Loop over the files in the folder
for filename in os.listdir(folder_path):
    if filename in file_mapping:
        df_name = file_mapping[filename]
        file_path = os.path.join(folder_path, filename)
        dataframes[df_name] = pd.read_csv(file_path, sep=',')

# Now you have DataFrames for the specified files
print(dataframes.keys())

/tmp/ipykernel_17045/1504379305.py:23: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes[df_name] = pd.read_csv(file_path, sep=',')


dict_keys(['stops_df', 'routes_df', 'stop_times_df', 'trips_df'])


/tmp/ipykernel_17045/1504379305.py:23: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes[df_name] = pd.read_csv(file_path, sep=',')


In [7]:
stop_times_df = dataframes["stop_times_df"][["trip_id","arrival_time","departure_time","stop_id","stop_sequence"]]
stops_df = dataframes["stops_df"][["stop_id","stop_name","stop_lat","stop_lon","stop_timezone"]]
# Calculate the mode of the "stop_timezone" column
mode_stop_timezone = stops_df['stop_timezone'].mode().iloc[0]
# Fill NaN values with the mode value
stops_df.loc[:, 'stop_timezone'].fillna(mode_stop_timezone, inplace=True)
trips_df = dataframes["trips_df"][["route_id", "trip_id"]].merge(dataframes["routes_df"][["route_id","route_short_name"]], on="route_id", how="left")
stop_times_df.loc[:, 'stop_id'] = stop_times_df['stop_id'].astype(str)


# Assuming you have dataframes named "stop_times_df" and "stops_df"
result_df = stop_times_df.merge(stops_df, on="stop_id", how="left")
result_df = result_df.merge(trips_df,on="trip_id", how="left")

result_df = result_df[["route_id","trip_id","arrival_time","stop_id","route_short_name"]]
result_df.loc[:, "city_name"] = city_name
# result_df

/tmp/ipykernel_17045/1373638198.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stops_df.loc[:, 'stop_timezone'].fillna(mode_stop_timezone, inplace=True)


In [7]:
# Get the current UTC time
current_utc_time = datetime.datetime.utcnow()
# Define the "Europe/Rome" timezone
timezone = pytz.timezone(mode_stop_timezone)
# Convert the current UTC time to "Europe/Rome" timezone
current_local_time = current_utc_time.replace(tzinfo=pytz.UTC).astimezone(timezone).strftime('%H:%M:%S')


In [51]:
# station_number = "12736"
# station_number = "47052"
station_number = input("Enter the station number: ")

In [56]:
x = (result_df[result_df['stop_id'].str.contains(station_number) & (result_df['arrival_time'] >= current_local_time)]
     .sort_values(by="arrival_time", ascending=True)
     .groupby('route_id').first()
     .sort_values(by="arrival_time", ascending=True)
     .reset_index()
     )

x

,route_id,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_name,stop_lat,stop_lon,stop_timezone,route_short_name
0,T01,1#3-2,11:38:58,11:38:58,74765,3,TUSCOLANA/CAMPO ROMANO,41.832195,12.615452,Europe/Rome,T01
1,506,1#2153-11,11:39:48,11:39:48,74765,29,TUSCOLANA/CAMPO ROMANO,41.832195,12.615452,Europe/Rome,506
2,509,0#2217-13,11:49:28,11:49:28,74765,11,TUSCOLANA/CAMPO ROMANO,41.832195,12.615452,Europe/Rome,509
3,T02,1#3116-2,11:52:32,11:52:32,74765,3,TUSCOLANA/CAMPO ROMANO,41.832195,12.615452,Europe/Rome,T02


In [74]:
dict(zip(x["route_short_name"], x["arrival_time"]))


{'T01': '11:38:58', '506': '11:39:48', '509': '11:49:28', 'T02': '11:52:32'}

In [ ]:
city_name = input("Enter the city name: ")
station_number = input("Enter the station number: ")

## Read data from the gtfs feed

In [7]:
import datetime
import pytz
import zipfile
import os
import pandas as pd

city_name = input("Enter the city name: ")
city_name = city_name.capitalize()

# Define the folder containing the txt files
folder_path = f'/home/elshan/Projects/GTFS/gtfs_unziped_data/{city_name}'

# Define a mapping from file names to DataFrame names
file_mapping = {
    "stop_times.txt": "stop_times_df",
    "stops.txt": "stops_df",
    "trips.txt": "trips_df",
    "routes.txt": "routes_df"
}

# Initialize a dictionary to store the DataFrames
dataframes = {}

# Loop over the files in the folder
for filename in os.listdir(folder_path):
    if filename in file_mapping:
        df_name = file_mapping[filename]
        file_path = os.path.join(folder_path, filename)
        dataframes[df_name] = pd.read_csv(file_path, sep=',')

# Now you have DataFrames for the specified files
print(dataframes.keys())

stop_times_df = dataframes["stop_times_df"][["trip_id","arrival_time","departure_time","stop_id","stop_sequence"]]
stops_df = dataframes["stops_df"][["stop_id","stop_name","stop_lat","stop_lon","stop_timezone"]]
# Calculate the mode of the "stop_timezone" column
mode_stop_timezone = stops_df['stop_timezone'].mode().iloc[0]
# Fill NaN values with the mode value
stops_df.loc[:, 'stop_timezone'].fillna(mode_stop_timezone, inplace=True)
trips_df = dataframes["trips_df"][["route_id", "trip_id"]].merge(dataframes["routes_df"][["route_id","route_short_name"]], on="route_id", how="left")
stop_times_df.loc[:, 'stop_id'] = stop_times_df['stop_id'].astype(str)


# Assuming you have dataframes named "stop_times_df" and "stops_df"
result_df = stop_times_df.merge(stops_df, on="stop_id", how="left")
result_df = result_df.merge(trips_df,on="trip_id", how="left")

result_df = result_df[["route_id","trip_id","arrival_time","stop_id","route_short_name","stop_timezone"]]
# result_df.loc[:, "city_name"] = city_name
# result_df

FileNotFoundError: [Errno 2] No such file or directory: '/home/elshan/Projects/GTFS/gtfs_unziped_data/Turin'

In [5]:
result_df.to_csv(f"/home/elshan/Projects/GTFS/gtfs_csv_files/{city_name}_gtfs.csv", index=False)

In [6]:
result_df

,route_id,trip_id,arrival_time,stop_id,route_short_name,stop_timezone
0,T01,1#1-2,07:28:00,74761,T01,Europe/Rome
1,T01,1#1-2,07:34:00,74763,T01,Europe/Rome
2,T01,1#1-2,07:36:44,74765,T01,Europe/Rome
3,T01,1#1-2,07:43:14,74716,T01,Europe/Rome
4,T01,1#1-2,07:51:00,82543,T01,Europe/Rome
...,...,...,...,...,...,...
4860085,2509,VJd7d824985a39d4bdbde2fb546891432d153e461b,19:52:52,74088,340,Europe/Rome
4860086,2509,VJd7d824985a39d4bdbde2fb546891432d153e461b,19:56:23,73845,340,Europe/Rome
4860087,2509,VJd7d824985a39d4bdbde2fb546891432d153e461b,19:58:09,73846,340,Europe/Rome
4860088,2509,VJd7d824985a39d4bdbde2fb546891432d153e461b,19:59:47,73847,340,Europe/Rome


In [ ]:
import mysql.connector

CREATE_GTFS_DATA_TABLE = (
    "CREATE TABLE IF NOT EXISTS avti_cb_test.gtfs_data (route_id VARCHAR(50), trip_id VARCHAR(60), arrival_time TIMESTAMP, stop_id VARCHAR(100), route_short_name VARCHAR(50), city_name VARCHAR(30))"
)

INSERT_IN_GTFS_DATA =(
    "INSERT INTO avti_cb_test.gtfs_data (route_id, trip_id, arrival_time, stop_id, route_short_name, city_name) VALUES (%s, %s, %s, %s, %s, %s)"
) 

# Establish a connection to the MySQL database
db_connection = mysql.connector.connect(
    user = "avti_cb_t",
    password = "KAUfuP0PvJfSS7Nc/+r1MQ=",
    host = "dtl-con-1.lunalabs.it",
    port = 3307,                                   # MySQL default port
    database = "avti_cb_test"
)

# Create a cursor
cursor = db_connection.cursor()

# Insert data in batches
chunksize = 1000  # Adjust this to your system's memory capacity
for i in range(0, len(result_df), chunksize):
    chunk = result_df[i:i + chunksize]
    data = [tuple(row) for row in chunk.values]

    cursor.executemany(INSERT_IN_GTFS_DATA, data)
    db_connection.commit()

# Close the cursor and the database connection
cursor.close()
db_connection.close()


In [5]:
paris_gtfs_df = pd.read_csv(f"/home/elshan/Projects/GTFS/gtfs_csv_files/{city_name}_gtfs.csv")
paris_gtfs_df

,route_id,trip_id,arrival_time,stop_id,route_short_name,city_name
0,IDFM:C01727,IDFM:TN:SNCF:a76d9836-c6b1-43d0-9d57-58fe52d5c6b7,10:38:10,IDFM:monomodalStopPlace:470540,C,Paris
1,IDFM:C01727,IDFM:TN:SNCF:a76d9836-c6b1-43d0-9d57-58fe52d5c6b7,10:39:50,IDFM:monomodalStopPlace:45705,C,Paris
2,IDFM:C01727,IDFM:TN:SNCF:a76d9836-c6b1-43d0-9d57-58fe52d5c6b7,10:43:00,IDFM:monomodalStopPlace:44877,C,Paris
3,IDFM:C01727,IDFM:TN:SNCF:a76d9836-c6b1-43d0-9d57-58fe52d5c6b7,10:47:00,IDFM:monomodalStopPlace:43072,C,Paris
4,IDFM:C01727,IDFM:TN:SNCF:a76d9836-c6b1-43d0-9d57-58fe52d5c6b7,10:49:50,IDFM:monomodalStopPlace:45301,C,Paris
...,...,...,...,...,...,...
9527923,IDFM:C01150,IDFM:RATP:113391-C01150-COU_RATP_5087260_27415...,17:09:00,IDFM:19890,121,Paris
9527924,IDFM:C01150,IDFM:RATP:113391-C01150-COU_RATP_5087260_27415...,17:11:00,IDFM:29462,121,Paris
9527925,IDFM:C01150,IDFM:RATP:113391-C01150-COU_RATP_5087260_27415...,17:14:00,IDFM:28521,121,Paris
9527926,IDFM:C01150,IDFM:RATP:113391-C01150-COU_RATP_5087260_27415...,17:17:00,IDFM:29463,121,Paris


In [8]:
# Get the current UTC time
current_utc_time = datetime.datetime.utcnow()
# Define the "Europe/Rome" timezone
timezone = pytz.timezone(mode_stop_timezone)
# Convert the current UTC time to "Europe/Rome" timezone
current_local_time = current_utc_time.replace(tzinfo=pytz.UTC).astimezone(timezone).strftime('%H:%M:%S')


station_number = "19890"
x = (paris_gtfs_df[paris_gtfs_df['stop_id'].str.contains(station_number) & (paris_gtfs_df['arrival_time'] >= current_local_time)]
     .sort_values(by="arrival_time", ascending=True)
     .groupby('route_id').first()
     .sort_values(by="arrival_time", ascending=True)
     .reset_index()
     )

x

,route_id,trip_id,arrival_time,stop_id,route_short_name,city_name
0,IDFM:C01150,IDFM:RATP:113391-C01150-COU_RATP_5087268_27472...,17:30:00,IDFM:19890,121,Paris
1,IDFM:C01398,IDFM:RATP:107674-C01398-COU_RATP_5083508_30535...,25:10:00,IDFM:19890,N34,Paris


In [ ]:
from datetime import datetime
import pytz
import zipfile
import os
import pandas as pd

def station_info_extraction(station_number,city_name= "Milan" ):    

    city_name = city_name.capitalize()

    result_df = pd.read_csv(f"/home/elshan/Projects/GTFS/gtfs_csv_files/{city_name}_gtfs.csv")

    # mode_stop_timezone= "Europe/Rome"
    mode_stop_timezone = result_df['stop_timezone'].mode().iloc[0]

    # Get the current UTC time
    current_utc_time = datetime.utcnow()
    # Define the "Europe/Rome" timezone
    timezone = pytz.timezone(mode_stop_timezone)
    # Convert the current UTC time to "Europe/Rome" timezone
    current_local_time = current_utc_time.replace(tzinfo=pytz.UTC).astimezone(timezone).strftime('%H:%M:%S')

    x = (result_df[result_df['stop_id'].str.contains(station_number) & (result_df['arrival_time'] >= current_local_time)]
     .sort_values(by="arrival_time", ascending=True)
     .groupby('route_id').first()
     .sort_values(by="arrival_time", ascending=True)
     .reset_index()
     )
    
    return dict(zip(x["route_short_name"], x["arrival_time"]))

In [2]:
station_info_extraction(city_name= "paris", station_number = "19890")

{'121': '16:57:00', 'N34': '25:10:00'}